In [ ]:
from grid.robot.airgen_car import AirGenCar
airgen_car_0 = AirGenCar(**{'geo': False})

Connected!
Client Ver:1 (Min Req: 1), Server Ver:1 (Min Req: 1)



In [ ]:
from grid.model.perception.segmentation.clipseg import CLIPSeg
seg_clipseg_0 = CLIPSeg()

preprocessor_config.json:   0%|          | 0.00/380 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/974 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.73k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/603M [00:00<?, ?B/s]

In [ ]:
import rerun as rr
import airgen
import time
client = airgen_car_0.client
client.enableApiControl(True)
client.setCarTargetSpeed(5)
time.sleep(10)
client.setCarTargetSpeed(0)
image = client.getImages("front_center", [0])
rr.log("Scene", rr.Image(image[0][0]))

In [ ]:
road = seg_clipseg_0.segment_image(image[0][0], "Road")
rr.log("Segmented Scene", rr.Image(road))

Unused or unrecognized kwargs: padding.


[2024-07-23T20:54:09Z WARN  re_log_types::path::parse_path] When parsing the entity path "Segmented Scene": Unescaped whitespace. The path will be interpreted as /Segmented\ Scene


In [ ]:
image_new = client.getImages("back_center", [0])
rr.log("Car", rr.Image(image_new[0][0]))

In [ ]:
car = seg_clipseg_0.segment_image(image_new[0][0], "Car")
rr.log("Segmented Car", rr.Image(car))

Unused or unrecognized kwargs: padding.


In [ ]:
client.setCarTargetSpeed(5)
while True:
    if client.simGetVehiclePose().position.x_val <= -120:
        client.setCarTargetSpeed(0)
        break
print(client.simGetVehiclePose().position)

<Vector3r> {   'x_val': -120.01766967773438,
    'y_val': -0.14030574262142181,
    'z_val': -0.2479163408279419}


In [ ]:
controls = client.getCarControls()
controls.is_manual_gear = False
controls.steering = 1.0
client.setCarControls(controls)
client.setCarTargetSpeed(2.1)
time.sleep(3)
client.setCarTargetSpeed(0)

In [ ]:
controls.steering = 0.0
client.setCarControls(controls)
client.setCarTargetSpeed(2)
time.sleep(3)
client.setCarTargetSpeed(0)

In [ ]:
def get_lidar_data(client):
    lidar_data = client.getLidarData()
    if len(lidar_data.point_cloud) < 3:
        print("No points received from Lidar")
    else:
        points = np.array(lidar_data.point_cloud, dtype=np.float32)
        points = np.reshape(points, (int(points.shape[0] / 3), 3))
        points_xyz = np.zeros_like(points)
        points_xyz[:, 0] = points[:, 0]
        points_xyz[:, 1] = points[:, 1]
        points_xyz[:, 2] = -points[:, 2] + 1
        print("Got LiDAR Data")
        return points_xyz

In [ ]:
%matplotlib inline
import os
from grid import GRID_USER_SESSION_BLOB_DIR
import numpy as np
import matplotlib.pyplot as plt
points = get_lidar_data(client)
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
for point in points:
    ax.scatter(point[0], point[1], point[2])
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
save_path = os.path.join(GRID_USER_SESSION_BLOB_DIR, "pcl.png")
plt.savefig(save_path, bbox_inches='tight')
print("Saved!")

In [ ]:
import cv2
img = cv2.imread(save_path)
rr.log("PCL", rr.Image(img))